## Quantization for LM

Reducing the size of Neural Networks is called Network Quantizaiton where the "quantization" meaning number of bits representing a number.

**nn.Module** vs **nn.Functional**
The technique we are going to use for quantization called "net-aware quantizaiton" is 1:1 mapping of operations and to model. So we cannot re-use operations.

In [1]:
import sklearn

In [2]:
from __future__ import absolute_import, division, print_function

import logging
import numpy as np
import os
import random
import sys
import time
import torch

from argparse import Namespace
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from tqdm import tqdm
from transformers import (BertConfig, BertForSequenceClassification, BertTokenizer,)
from transformers import glue_compute_metrics as compute_metrics
from transformers import glue_output_modes as output_modes
from transformers import glue_processors as processors
from transformers import glue_convert_examples_to_features as convert_examples_to_features

# Setup logging
logger = logging.getLogger(__name__)
logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.WARN)

logging.getLogger("transformers.modeling_utils").setLevel(
   logging.WARN)  # Reduce logging

print(torch.__version__)

1.4.0


In [3]:
torch.set_num_threads(1)
print(torch.__config__.parallel_info())

ATen/Parallel:
	at::get_num_threads() : 1
	at::get_num_interop_threads() : 4
OpenMP not found
Intel(R) Math Kernel Library Version 2019.0.5 Product Build 20190808 for Intel(R) 64 architecture applications
	mkl_get_max_threads() : 1
Intel(R) MKL-DNN v0.21.1 (Git Hash 7d2fd500bc78936d1d648ca713b901012f470dbc)
std::thread::hardware_concurrency() : 8
Environment variables:
	OMP_NUM_THREADS : [not set]
	MKL_NUM_THREADS : [not set]
ATen parallel backend: OpenMP



We load wikitext from the torchtext module



In [6]:
class Dictionary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []
        
    def add_word(self, word):
        if word not in self.word2idx.keys():
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) -1
        return self.word2idx[word]
    def __len__(self):
        return len(self.idx2word)
    
class Corpus(object):
    def __init__(self, path):
        self.dictionary = Dictionary()
        self.train = self.tokenize(os.path.join(path, 'train.txt'))
        self.valid = self.tokenize(os.path.join(path, 'valid.txt'))
        self.test = self.tokenize(os.path.join(path, 'test.txt'))
    
    def tokenize(self, path):
        
        assert os.path.exists(path)
        
        with open(path, 'r', encoding='utf-8') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    self.dictionary.add_word(word)
        with open(path, 'r', encoding="utf8") as f:
            idss = []
            for line in f:
                words = line.split() + ['<eos>']
                ids = []
                for word in words:
                    ids.append(self.dictionary.word2idx[word])
                    
                idss.append(torch.tensor(ids).type(torch.int64))
            ids = torch.cat(idss)
        return ids
    
model_data_filepath = '../'

corpus = Corpus('../wikitext-2')
                    
 
        
            

In [9]:
ntokens = len(corpus.dictionary)

model = LSTMModel(
    ntoken = ntokens,
    ninp = 512,
    nhid = 256,
    nlayers = 5,
)

model.load_state_dict(
    torch.load(
        'word_language_model_quantize.pth',
        map_location=torch.device('cpu')
        )
    )

model.eval()
print(model)

RuntimeError: Error(s) in loading state_dict for LSTMModel:
	size mismatch for encoder.weight: copying a param with shape torch.Size([33278, 512]) from checkpoint, the shape in current model is torch.Size([18328, 512]).
	size mismatch for decoder.weight: copying a param with shape torch.Size([33278, 256]) from checkpoint, the shape in current model is torch.Size([18328, 256]).
	size mismatch for decoder.bias: copying a param with shape torch.Size([33278]) from checkpoint, the shape in current model is torch.Size([18328]).

In [ ]:
input = torch.randint(ntokens, (1,1), dtype=torch.)